In [1]:
import torch
from transformers import BertTokenizer, BertForSequenceClassification
import random

# --- Check available device ---
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# --- Load Model 1 (The "Sentiment Detector") ---
print("Loading Model 1 (Neg vs. Not-Neg)...")
model_1_path = "./bert_model_1_neg_vs_notneg"
tokenizer_1 = BertTokenizer.from_pretrained(model_1_path)
model_1 = BertForSequenceClassification.from_pretrained(model_1_path).to(device)
model_1.eval() # Set to evaluation mode

# --- Load Model 2 (The "Nuance Detector") ---
print("Loading Model 2 (Neu vs. Pos)...")
model_2_path = "./bert_model_2_neu_vs_pos"
tokenizer_2 = BertTokenizer.from_pretrained(model_2_path)
model_2 = BertForSequenceClassification.from_pretrained(model_2_path).to(device)
model_2.eval() # Set to evaluation mode

print("All models loaded!")

c:\Users\Sourabh\anaconda3\envs\news_sentiment\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Using device: cuda
Loading Model 1 (Neg vs. Not-Neg)...
Loading Model 2 (Neu vs. Pos)...
All models loaded!


In [2]:
def predict_sentiment(text: str):
    """
    Runs the hierarchical prediction:
    1. Check if Negative vs. Not-Negative (using Model 1).
    2. If Not-Negative, check if Neutral vs. Positive (using Model 2).
    """
    
    # --- Step 1: Feed text to Model 1 ---
    inputs_1 = tokenizer_1(text, return_tensors="pt", padding=True, truncation=True, max_length=512)
    inputs_1 = {k: v.to(device) for k, v in inputs_1.items()}
    
    with torch.no_grad():
        outputs_1 = model_1(**inputs_1)
    
    pred_1 = torch.argmax(outputs_1.logits, dim=1).item()
    
    # --- Step 2: Check Model 1's prediction ---
    if pred_1 == 0:
        # Model 1 predicted "Negative"
        # We can even get its confidence
        probs_1 = torch.nn.functional.softmax(outputs_1.logits, dim=-1)
        confidence = probs_1[0][pred_1].item()
        return {"sentiment": "Negative", "confidence": round(confidence, 4)}
    
    else:
        # Model 1 predicted "Not-Negative" (1), so we run Model 2
        
        # --- Step 3: Feed text to Model 2 ---
        inputs_2 = tokenizer_2(text, return_tensors="pt", padding=True, truncation=True, max_length=512)
        inputs_2 = {k: v.to(device) for k, v in inputs_2.items()}
    
        with torch.no_grad():
            outputs_2 = model_2(**inputs_2)
            
        pred_2 = torch.argmax(outputs_2.logits, dim=1).item()
        probs_2 = torch.nn.functional.softmax(outputs_2.logits, dim=-1)
        confidence = probs_2[0][pred_2].item()
        
        # --- Step 4: Check Model 2's prediction ---
        if pred_2 == 0:
            return {"sentiment": "Neutral", "confidence": round(confidence, 4)}
        else: # pred_2 == 1
            return {"sentiment": "Positive", "confidence": round(confidence, 4)}

In [3]:
# --- Example prediction ---
text_1 = "The economy is booming and people are happy!"
text_2 = "The company announced terrible earnings."
text_3 = "The meeting will be held on Tuesday."

print(f"'{text_1}':\n {predict_sentiment(text_1)}\n")
print(f"'{text_2}':\n {predict_sentiment(text_2)}\n")
print(f"'{text_3}':\n {predict_sentiment(text_3)}\n")

'The economy is booming and people are happy!':
 {'sentiment': 'Positive', 'confidence': 0.8463}

'The company announced terrible earnings.':
 {'sentiment': 'Negative', 'confidence': 0.9998}

'The meeting will be held on Tuesday.':
 {'sentiment': 'Neutral', 'confidence': 0.9988}

